In [3]:
import pandas as pd

df = pd.read_csv("cleaned_data.csv")

df.head()


,text,label,language
0,ওই হালার পুত এখন কি মদ খাওয়ার সময় রাতের বেলা...,Not Approved,BN
1,আপনার জন্ম প্রক্রিয়ার সময় আপনার মায়ের ভিতর কি ...,Not Approved,BN
2,ধজভংগ দের আর ভায়াগ্রা লাগবো না। ধংস হোক এই সব ...,Not Approved,BN
3,বোকাচোদা একটা।,Not Approved,BN
4,তোর দেশে ফেরার অপেক্ষায় রইলাম। জেলে একটা কামরা...,Not Approved,BN


In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])


In [5]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(),
    df['label_encoded'].tolist(),
    test_size=0.2,
    random_state=42
)


In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


/home/asif/.pyenv/versions/tf-bert-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
/home/asif/.pyenv/versions/tf-bert-py310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
def tokenize(texts):
    return tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='tf'
    )

train_inputs = tokenize(train_texts)
test_inputs = tokenize(test_texts)


2025-12-24 19:11:36.888665: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-24 19:11:36.993607: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-24 19:11:37.238735: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-24 19:11:37.238787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-24 19:11:37.258581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [ ]:
from transformers import TFBertModel
import tensorflow as tf

bert = TFBertModel.from_pretrained('bert-base-multilingual-cased')

input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

outputs = bert(input_ids, attention_mask=attention_mask)
cls_output = outputs.last_hidden_state[:, 0, :]  

x = tf.keras.layers.Dense(256, activation='relu')(cls_output)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
history = model.fit(
    {
        "input_ids": train_inputs["input_ids"],
        "attention_mask": train_inputs["attention_mask"]
    },
    train_labels,
    validation_split=0.1,
    epochs=3,
    batch_size=8
)


In [ ]:
model.evaluate(
    {
        "input_ids": test_inputs["input_ids"],
        "attention_mask": test_inputs["attention_mask"]
    },
    test_labels
)
